In [1]:
# 📦 1. Importar librerías
import pandas as pd

# 📂 2. Cargar el archivo CSV desde tu máquina local
ruta = r"C:\Users\waldo\Dropbox\Maestría Ciencia de Datos\Labo 3\Proceso\finalitoul.csv"
df = pd.read_csv(ruta)

# 📊 3. Mostrar dimensiones y tipos de datos
print("🔢 Shape del DataFrame:", df.shape)
print("\n🧱 Tipos de columnas:")
print(df.dtypes)

# 👀 4. Vista previa de los datos
print("\n🔍 Primeras filas del dataset:")
display(df.head())


🔢 Shape del DataFrame: (16763760, 13)

🧱 Tipos de columnas:
customer_id                int64
product_id                 int64
periodo                   object
plan_precios_cuidados    float64
cust_request_qty         float64
cust_request_tn          float64
tn                       float64
stock_final              float64
cat1                      object
cat2                      object
cat3                      object
brand                     object
sku_size                   int64
dtype: object

🔍 Primeras filas del dataset:


,customer_id,product_id,periodo,plan_precios_cuidados,cust_request_qty,cust_request_tn,tn,stock_final,cat1,cat2,cat3,brand,sku_size
0,10001,20001,2017-01-01,0.0,11.0,99.43861,99.43861,0.0,HC,ROPA LAVADO,Liquido,ARIEL,3000
1,10001,20001,2017-02-01,0.0,23.0,198.84365,198.84365,0.0,HC,ROPA LAVADO,Liquido,ARIEL,3000
2,10001,20001,2017-03-01,0.0,33.0,92.46537,92.46537,0.0,HC,ROPA LAVADO,Liquido,ARIEL,3000
3,10001,20001,2017-04-01,0.0,8.0,13.29728,13.29728,0.0,HC,ROPA LAVADO,Liquido,ARIEL,3000
4,10001,20001,2017-05-01,0.0,15.0,101.20711,101.00563,0.0,HC,ROPA LAVADO,Liquido,ARIEL,3000


In [2]:
# 🧮 1. Cuántos períodos distintos hay
n_periodos = df['periodo'].nunique()
print(f"📆 Períodos únicos en el dataset: {n_periodos}")

# 🧲 2. Agrupar por combinación (customer_id, product_id)
agrupado = df.groupby(['customer_id', 'product_id'])

# 📐 3. Filtrar combinaciones con 36 períodos y suma de tn == 0
combinaciones_sin_ventas = agrupado.agg(
    cantidad_periodos=('periodo', 'nunique'),
    suma_tn=('tn', 'sum')
).reset_index()

# 🧹 4. Filtrar los que tienen todos los períodos y nunca vendieron
combinaciones_sin_ventas = combinaciones_sin_ventas[
    (combinaciones_sin_ventas['cantidad_periodos'] == n_periodos) &
    (combinaciones_sin_ventas['suma_tn'] == 0)
]

print(f"🔎 Combinaciones sin venta en los {n_periodos} períodos:", combinaciones_sin_ventas.shape[0])

# 📂 5. Si querés filtrar el dataset original para ver esos registros:
df_sin_ventas = df.merge(
    combinaciones_sin_ventas[['customer_id', 'product_id']],
    on=['customer_id', 'product_id'],
    how='inner'
)


📆 Períodos únicos en el dataset: 36
🔎 Combinaciones sin venta en los 36 períodos: 202855


In [3]:
# 🧹 6. Eliminar esas combinaciones del dataset original
df_filtrado = df.merge(
    combinaciones_sin_ventas[['customer_id', 'product_id']],
    on=['customer_id', 'product_id'],
    how='left',
    indicator=True
)

# Nos quedamos con las filas que NO estaban en las combinaciones sin ventas
df_filtrado = df_filtrado[df_filtrado['_merge'] == 'left_only'].drop(columns=['_merge'])

print("📉 Shape después de eliminar combinaciones sin venta:", df_filtrado.shape)

# 💾 7. Guardar el nuevo CSV (ajustá la ruta si querés)
ruta_salida = r"C:\Users\waldo\Dropbox\Maestría Ciencia de Datos\Labo 3\Proceso\finalitoul_filtrado.csv"
df_filtrado.to_csv(ruta_salida, index=False)

print(f"✅ Archivo guardado en: {ruta_salida}")


📉 Shape después de eliminar combinaciones sin venta: (9460980, 13)
✅ Archivo guardado en: C:\Users\waldo\Dropbox\Maestría Ciencia de Datos\Labo 3\Proceso\finalitoul_filtrado.csv


In [4]:
# 📊 Métricas del proceso de filtrado

# Total de combinaciones originales
total_combinaciones = df.groupby(['customer_id', 'product_id']).ngroups

# Combinaciones eliminadas
combinaciones_eliminadas = combinaciones_sin_ventas.shape[0]

# Combinaciones restantes
combinaciones_restantes = total_combinaciones - combinaciones_eliminadas

# % de combinaciones eliminadas
porcentaje_eliminado = (combinaciones_eliminadas / total_combinaciones) * 100

# % de filas eliminadas
filas_eliminadas = df.shape[0] - df_filtrado.shape[0]
porcentaje_filas = (filas_eliminadas / df.shape[0]) * 100

# 📋 Reporte
print("\n📋 RESUMEN:")
print(f"🔸 Total de combinaciones originales: {total_combinaciones:,}")
print(f"🔻 Combinaciones eliminadas: {combinaciones_eliminadas:,} ({porcentaje_eliminado:.2f}%)")
print(f"✅ Combinaciones restantes: {combinaciones_restantes:,}")
print(f"📉 Filas eliminadas: {filas_eliminadas:,} ({porcentaje_filas:.2f}%)")
print(f"💾 Dataset final guardado en: {ruta_salida}")



📋 RESUMEN:
🔸 Total de combinaciones originales: 465,660
🔻 Combinaciones eliminadas: 202,855 (43.56%)
✅ Combinaciones restantes: 262,805
📉 Filas eliminadas: 7,302,780 (43.56%)
💾 Dataset final guardado en: C:\Users\waldo\Dropbox\Maestría Ciencia de Datos\Labo 3\Proceso\finalitoul_filtrado.csv


In [5]:
# 💾 Guardar combinaciones eliminadas para trazabilidad
ruta_combinaciones = r"C:\Users\waldo\Dropbox\Maestría Ciencia de Datos\Labo 3\Proceso\combinaciones_eliminadas.csv"
combinaciones_sin_ventas.to_csv(ruta_combinaciones, index=False)

print(f"📁 Combinaciones eliminadas guardadas en: {ruta_combinaciones}")


📁 Combinaciones eliminadas guardadas en: C:\Users\waldo\Dropbox\Maestría Ciencia de Datos\Labo 3\Proceso\combinaciones_eliminadas.csv


In [6]:
# 📂 Cargar el archivo con combinaciones eliminadas
df_eliminadas = pd.read_csv(ruta_combinaciones)

# 👀 Mostrar primeras filas
print("🔍 Primeras combinaciones eliminadas:")
display(df_eliminadas.head())

# 🧮 ¿Cuántas combinaciones hay?
print(f"🔢 Total de combinaciones eliminadas: {df_eliminadas.shape[0]}")

# 📊 Si querés ver qué customers o productos aparecen más en este subconjunto:
top_customers = df_eliminadas['customer_id'].value_counts().head(10)
top_products = df_eliminadas['product_id'].value_counts().head(10)

print("\n👤 Top 10 customers más frecuentes entre los eliminados:")
print(top_customers)

print("\n📦 Top 10 productos más frecuentes entre los eliminados:")
print(top_products)


🔍 Primeras combinaciones eliminadas:


,customer_id,product_id,cantidad_periodos,suma_tn
0,10001,20032,36,0.0
1,10001,20052,36,0.0
2,10001,20069,36,0.0
3,10001,20071,36,0.0
4,10001,20089,36,0.0


🔢 Total de combinaciones eliminadas: 202855

👤 Top 10 customers más frecuentes entre los eliminados:
customer_id
10632    778
10633    778
10637    778
10559    777
10373    777
10636    777
10577    776
10591    776
10558    775
10507    775
Name: count, dtype: int64

📦 Top 10 productos más frecuentes entre los eliminados:
product_id
21252    575
21267    574
21276    574
20953    550
20845    546
20886    545
21266    537
21265    537
21246    531
21262    531
Name: count, dtype: int64


In [7]:
# 📂 Cargar el dataset original (si no lo tenés en memoria)
ruta_original = r"C:\Users\waldo\Dropbox\Maestría Ciencia de Datos\Labo 3\Proceso\finalitoul.csv"
df = pd.read_csv(ruta_original)

# 📂 Cargar las combinaciones eliminadas
ruta_combinaciones = r"C:\Users\waldo\Dropbox\Maestría Ciencia de Datos\Labo 3\Proceso\combinaciones_eliminadas.csv"
df_eliminadas = pd.read_csv(ruta_combinaciones)

# 🔄 Hacer un merge para reconstruir todos los registros que fueron eliminados
df_eliminados_completo = df.merge(
    df_eliminadas,
    on=['customer_id', 'product_id'],
    how='inner'
).sort_values(['customer_id', 'product_id', 'periodo'])

# 👀 Mostrar una vista general
print("🧾 Registros eliminados reconstruidos para inspección:")
display(df_eliminados_completo.head())


🧾 Registros eliminados reconstruidos para inspección:


,customer_id,product_id,periodo,plan_precios_cuidados,cust_request_qty,cust_request_tn,tn,stock_final,cat1,cat2,cat3,brand,sku_size,cantidad_periodos,suma_tn
0,10001,20032,2017-01-01,0.0,0.0,0.0,0.0,0.0,HC,ROPA LAVADO,Liquido,ROPEX2,3000,36,0.0
1,10001,20032,2017-02-01,0.0,0.0,0.0,0.0,0.0,HC,ROPA LAVADO,Liquido,ROPEX2,3000,36,0.0
2,10001,20032,2017-03-01,0.0,0.0,0.0,0.0,0.0,HC,ROPA LAVADO,Liquido,ROPEX2,3000,36,0.0
3,10001,20032,2017-04-01,0.0,0.0,0.0,0.0,0.0,HC,ROPA LAVADO,Liquido,ROPEX2,3000,36,0.0
4,10001,20032,2017-05-01,0.0,0.0,0.0,0.0,0.0,HC,ROPA LAVADO,Liquido,ROPEX2,3000,36,0.0


In [ ]:
df_eliminados_completo

In [8]:
ruta_inspeccion = r"C:\Users\waldo\Dropbox\Maestría Ciencia de Datos\Labo 3\Proceso\finalitoul_inspeccion_eliminados.csv"
df_eliminados_completo.to_csv(ruta_inspeccion, index=False)

print(f"📁 Archivo de inspección guardado en:\n{ruta_inspeccion}")


📁 Archivo de inspección guardado en:
C:\Users\waldo\Dropbox\Maestría Ciencia de Datos\Labo 3\Proceso\finalitoul_inspeccion_eliminados.csv


In [ ]:
# Si querés volver a cargarlo:
df_filtrado = pd.read_csv(ruta_salida)
df_filtrado